In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model-finetuned/finetuned_model/config.json
/kaggle/input/model-finetuned/finetuned_model/model.safetensors
/kaggle/input/model-finetuned/finetuned_model/checkpoint-13000/config.json
/kaggle/input/model-finetuned/finetuned_model/checkpoint-13000/trainer_state.json
/kaggle/input/model-finetuned/finetuned_model/checkpoint-13000/training_args.bin
/kaggle/input/model-finetuned/finetuned_model/checkpoint-13000/scheduler.pt
/kaggle/input/model-finetuned/finetuned_model/checkpoint-13000/model.safetensors
/kaggle/input/model-finetuned/finetuned_model/checkpoint-13000/optimizer.pt
/kaggle/input/model-finetuned/finetuned_model/checkpoint-13000/rng_state.pth
/kaggle/input/model-finetuned/finetuned_model/checkpoint-12500/config.json
/kaggle/input/model-finetuned/finetuned_model/checkpoint-12500/trainer_state.json
/kaggle/input/model-finetuned/finetuned_model/checkpoint-12500/training_args.bin
/kaggle/input/model-finetuned/finetuned_model/checkpoint-12500/scheduler.pt
/kaggle/input/mo

In [2]:
!pip install transformers torch

In [3]:
!pip install accelerate -U

In [4]:
import pandas as pd
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
train = pd.read_csv("/kaggle/input/daigt-one-place-all-data/concatenated.csv", sep=',')
val = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')

In [5]:
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)

In [6]:
val = val.drop_duplicates(subset=['text'])
val.reset_index(drop=True, inplace=True)

In [7]:
train.head()

,id,prompt_id,text,generated,model,kaggle_repo
0,d429f032,0,Advantages of Limiting Car Usage \n\nLimiting ...,1,gpt-3.5-turbo,1
1,1ce279be,0,Advantages of Limiting Car Usage\n\nLimiting c...,1,gpt-3.5-turbo,1
2,c9595213,0,Limiting car usage has numerous advantages tha...,1,gpt-3.5-turbo,1
3,f2266d87,0,The passages provided discuss the advantages o...,1,gpt-3.5-turbo,1
4,eeace4bd,0,Title: The Advantages of Limiting Car Usage\n\...,1,gpt-3.5-turbo,1


In [8]:
train.shape

(53729, 6)

In [9]:
test.head()

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [10]:
val.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [11]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
from transformers import AutoTokenizer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/tokenizer/tokenizer")

In [13]:
# Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("andreas122001/bloomz-560m-mixed-detector")
# model = AutoModelForSequenceClassification.from_pretrained("andreas122001/bloomz-560m-mixed-detector")

In [14]:
max_length = 512
#Tokenize the datasets with the specified maximum length
# train_encodings = tokenizer(train['text'].tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='pt')
# eval_encodings = tokenizer(val['text'].tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(test['text'].tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='pt')

In [15]:
# Convert labels to tensors
# train_labels = torch.tensor(train['generated'].tolist())
# eval_labels = torch.tensor(val['generated'].tolist())
test_labels = [0]*len(test)

In [16]:
# Create PyTorch datasets
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
# train_dataset = Dataset(train_encodings, train_labels)
# eval_dataset = Dataset(eval_encodings, eval_labels)
test_dataset = Dataset(test_encodings, test_labels)

In [17]:
# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./finetuned_model",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=1,
#     save_steps=500,
#     save_total_limit=2,
#     learning_rate=2e-5,
#     report_to=[]
# )

In [18]:
# # Define Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset
# )

In [19]:
# # Fine-tune the model
# trainer.train()

In [20]:
# # Save the fine-tuned model
# model.save_pretrained("./finetuned_model")

In [21]:
# Load the fine-tuned model
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/model-finetuned/finetuned_model")

In [22]:
# Create PyTorch dataloaders
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [23]:
# Put the model in evaluation mode
model.eval()

BloomForSequenceClassification(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [24]:
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model.to(device)

BloomForSequenceClassification(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [25]:
import torch.nn.functional as F
import numpy as np
# Lists to store predictions and true labels
all_predictions = []
all_true_labels = []

# Iterate over the test dataset batches
for batch in test_dataloader:
    # Move input tensors to the device (e.g., GPU if available)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    # Make predictions
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask).logits

    # Apply softmax to get probabilities
    probabilities = F.softmax(logits, dim=1)

    # Assuming binary classification, extract probability for the positive class (generated)
    generated_probabilities = probabilities[:, 1].cpu().numpy()

    # Append predictions and true labels to the lists
    all_predictions.extend(generated_probabilities)
    all_true_labels.extend(batch['labels'].numpy())

# Convert the lists to NumPy arrays
all_predictions = np.array(all_predictions)
all_true_labels = np.array(all_true_labels)

all_true_labels

array([0, 0, 0])

In [26]:
sub['generated'] = all_true_labels
sub.to_csv('submission.csv', index=False)
sub

,id,generated
0,0000aaaa,0
1,1111bbbb,0
2,2222cccc,0
